In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np
from scipy import linalg as la

np.set_printoptions(precision=3, linewidth=120)

# Real valued GHF

## Molecular setup

GHF SCF calculations start off in the same way as RHF SCF calculations. We first need a second-quantized Hamiltonian that expresses all the integrals in the AO basis.

In [2]:
molecule = gqcpy.Molecule.HRingFromDistance(3, 1.0, 0)  # create a neutral molecule
N = molecule.numberOfElectrons()

spinor_basis = gqcpy.GSpinorBasis(molecule, "STO-3G")
S = spinor_basis.quantizeOverlapOperator().parameters()

sq_hamiltonian = gqcpy.SQHamiltonian.Molecular(spinor_basis, molecule)  # 'sq' for 'second-quantized'

## The SCF procedure

We need an GHF SCF solver and an GHF SCF environment.

In [3]:
environment = gqcpy.GHFSCFEnvironment_d.WithCoreGuess(N, sq_hamiltonian, S)
plain_solver = gqcpy.GHFSCFSolver_d.Plain(threshold=1.0e-04, maximum_number_of_iterations=1000)  

We can use the solver to optimize the UHF parameters.

In [4]:
qc_structure = gqcpy.GHF_d.optimize(plain_solver, environment)

Check the resulting energy.

In [5]:
print(qc_structure.groundStateEnergy() + gqcpy.Operator.NuclearRepulsion(molecule).value())

-0.631146307416838


The problem here is, that we find a UHF solution. In order to find a true GHF solution using real valued GHF is to start from a random guess.

In [6]:
dimension = spinor_basis.numberOfSpinors()
np.random.seed(2)
x = np.random.rand(dimension, dimension)

# Make the matrix symmetric by adding it's transpose.
# Get the eigenvectors to use them, since they form a unitary matrix.
x_t = x.T
y = x + x_t
val, vec = la.eigh(y)

In [7]:
environment_random = gqcpy.GHFSCFEnvironment_d(N, sq_hamiltonian, S, vec)
plain_solver_random = gqcpy.GHFSCFSolver_d.Plain(threshold=1.0e-04, maximum_number_of_iterations=5000)
qc_structure_random = gqcpy.GHF_d.optimize(plain_solver_random, environment_random)

In [8]:
print(qc_structure_random.groundStateEnergy() + gqcpy.Operator.NuclearRepulsion(molecule).value())

-0.6318365333996345


# Complex valued GHF

Another way to converge to the true GHF solution is by breaking complex conjugation symmetry: aka working with complex valued orbitals. Let's start by adding a small complex perturbation to the core Hamiltonian.

In [9]:
complex_guess = sq_hamiltonian.core().parameters()
complex_guess = complex_guess + 0j
complex_guess[0, :] += 0.1j
complex_guess[:, 0] -= 0.1j

In [10]:
environment_complex = gqcpy.GHFSCFEnvironment_cd(N, sq_hamiltonian, S, complex_guess)
plain_solver_complex = gqcpy.GHFSCFSolver_cd.Plain(threshold=1.0e-04, maximum_number_of_iterations=5000)
qc_structure_complex = gqcpy.GHF_cd.optimize(plain_solver_complex, environment_complex)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. gqcpy.GHFSCFEnvironment_cd(N: int, sq_hamiltonian: GQCP::SQHamiltonian<std::__1::complex<double> >, S: numpy.ndarray[float64[m, n]], C_init: numpy.ndarray[complex128[m, n]])

Invoked with: 3, <gqcpy.SQHamiltonian object at 0x7fc278b57ef0>, array([[1.   , 0.797, 0.797, 0.   , 0.   , 0.   ],
       [0.797, 1.   , 0.797, 0.   , 0.   , 0.   ],
       [0.797, 0.797, 1.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 1.   , 0.797, 0.797],
       [0.   , 0.   , 0.   , 0.797, 1.   , 0.797],
       [0.   , 0.   , 0.   , 0.797, 0.797, 1.   ]]), array([[-2.09 +0.j , -1.885+0.1j, -1.885+0.1j,  0.   +0.1j,  0.   +0.1j,  0.   +0.1j],
       [-1.885-0.1j, -2.09 +0.j , -1.885+0.j ,  0.   +0.j ,  0.   +0.j ,  0.   +0.j ],
       [-1.885-0.1j, -1.885+0.j , -2.09 +0.j ,  0.   +0.j ,  0.   +0.j ,  0.   +0.j ],
       [ 0.   -0.1j,  0.   +0.j ,  0.   +0.j , -2.09 +0.j , -1.885+0.j , -1.885+0